In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [21]:

import os
import glob
import sys
import multiprocessing as mpp
import denswavemovie as dwp

reduce_factor=1
new_size=800
delay=20
min_frame = 5
max_frame = 600

mpp_mode=True

### Make the images
   (do we need to first make the images? run this script; it will save the files where they belong.)

In [13]:
make_images = False
#make_images = True
#
# this probably needs to be multiprocessed as well...
print('begin making movie images...')
if make_images:
    aa = dwp.johnsMovies(dosave=False,figsize=(6,5), legend_font_size=10)
#
print('completed: movie images made')

begin making movie images...
completed: movie images made


### Earthquake version:
(wrap in function calls so we can mpp them).

In [15]:
def do_eq():
    files_list = sorted([g for j,g in enumerate(glob.glob('imagesEq/*.png')) if (int(g.split('-')[1].split('.')[0])<max_frame 
                         and int(g.split('-')[1].split('.')[0])>=min_frame)],
                        key=lambda rw: int(rw.split('-')[1].split('.')[0]))[::reduce_factor]
    print(files_list[0:5])


    convert_str = 'convert -delay {} -loop 0 -resize {}x {} animated_dwf_eq.gif'.format(delay, new_size, 
        ' '.join(files_list)) 

    #print(convert_str)
    os.system(convert_str)

### Econ version:

In [17]:
def do_econ():
    files_list = sorted([g for j,g in enumerate(glob.glob('imagesEcon/*.png'))
                         if (int(g.split('-')[1].split('.')[0])<max_frame 
                         and int(g.split('-')[1].split('.')[0])>=min_frame)],
                        key=lambda rw: int(rw.split('-')[1].split('.')[0]))[::reduce_factor]
    print(files_list[0:5])


    convert_str = 'convert -delay {} -loop 0 -resize {}x {} animated_dwf_econ.gif'.format(delay, new_size, 
        ' '.join(files_list)) 

    #print(convert_str)
    os.system(convert_str)

### MPP (python) runner:
(actaully, i think "convert" is multi-threaded, so this doesn't really do anything for us.

In [24]:
if mpp_mode:
    # ... and if this gets any more complicated, we'll want a Pool() or something...
    # and note that it is possible that this will go badly becaust the matplotlib backents might interfere.
    # (except this scritp assumes mpl is already done... which means, i think, that "convert" is already threaded, right?
    # so we don't need to mpp here?)
    #
    P1 = mpp.Process(target=do_eq)
    P2 = mpp.Process(target=do_econ)

    P1.start()
    print('P1 started.')
    P2.start()
    print('P2 started.')
    #
    # they don't return anything, so we don't really need to join, but it won't hurt...
    P1.join()
    print('finished P1 join()')
    P2.join()
    print('finishde P2 join()')

    print('we should be finished...')



P1 started.
['imagesEq/dwaveFrame-0005.png', 'imagesEq/dwaveFrame-0006.png', 'imagesEq/dwaveFrame-0007.png', 'imagesEq/dwaveFrame-0008.png', 'imagesEq/dwaveFrame-0009.png']
['imagesEcon/dwaveFrame-0005.png', 'imagesEcon/dwaveFrame-0006.png', 'imagesEcon/dwaveFrame-0007.png', 'imagesEcon/dwaveFrame-0008.png', 'imagesEcon/dwaveFrame-0009.png']
P2 started.
finished P1 join()
finishde P2 join()
we should be finished...


### Serial runner
( and i think "convert" does good multi-threading )

In [ ]:
if not mpp_mode:
    print('starting...')
    print('doing_eq()')
    a=do_eq()

    print('doing econ()...')
    b=do_econ()
    print('finished.')